**1. Данные из таблички**

In [1]:
import pandas as pd

In [2]:
data_all = pd.read_excel("./friends_full_data_checked.xlsx")

In [3]:
data_all = data_all.drop("Unnamed: 0", axis=1)

In [4]:
data_all.tail()

,season,episode,monica_lines,joey_lines,chandler_lines,phoebe_lines,ross_lines,rachel_lines,scriptwriters
222,10,13,"Yeah, you really shouldn't. By the way, how ...","Well anyway, I'm glad you're back, I really n...","So, where's Mike? Hey. Why are you wearing my...","Hi Oh, he's at the doctor, he didn't poop the...","Joey, you shouldn't lie on your résumé. It w...",How was the honeymoon? Oh! I did not know you...,"Sherry Bilsing-Graham, Ellen Plummer"
223,10,14,"Well, because every time we do, you make joke...",Guess what? I finally got that seed out of ...,"Yeah, I don't know why we hang out with marri...","Oh! Yeah, this is fun, couples night. Wow! I ...","Hey, you guys... I have great news. Oh, well...","Hi you guys. Ooh, Italian! Hey you guys......",Tracy Reilly
224,10,15,"Hey, this afternoon you said you'd be support...","Hey, great! All right! What? No, no, no! No, ...",How did the job stuff go? Or facing a bitch o...,"Ooh, what's going on? Good for you! Of course...",You know what? This calls for a bottle of Is...,"He offered me one. The job is in Paris. Oh, ...","Marta Kauffman, David Crane"
225,10,16,"Hey, Rach, you're leaving tomorrow, shouldn't...","All right, all right, all right, let's play o...",Hey! So we thought we'd throw you little goin...,"Hey! Oh, he has a gig. I kinda like being mar...","Hey! Hey Erica, welcome back to town! Wow, ...","Ok! Can't believe I'm risking this again, bu...","Andrew Reich, Ted Cohen"
226,10,17,"Erica, are you okay? Oh my God! She doesn't h...",Hey! It's my house-warming present for Monica...,Relax! We'll just get her some antacids. Oh m...,Morning. What's that? It's a baby chick and d...,"Rach! Hey. Oh. This was amazing. Yeah, well, ...",So if you think I didn't say goodbye to you b...,"Marta Kauffman, David Crane"


**2. Модули и модели для парсинга**

In [5]:
! python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 2.6 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
from tqdm import tqdm

In [7]:
import re

In [8]:
import spacy
from spacy.lang.en import English

# для разбиения на предложения (модель слишком умная)
nlp_sent_segm = English()
nlp_sent_segm.add_pipe("sentencizer")

# для постеггинга и лемматизации
nlp_pos = spacy.load("en_core_web_sm")

In [9]:
nlp_pos("him")[0].lemma_

'he'

**3. Разбиение на предложения**

In [10]:
spaces = re.compile(r'\s{2,}')
def sent_segm(episode):
    d = spaces.sub(' ', episode)
    d = d.replace("’", "'")
    d = nlp_sent_segm(d)
    return d

In [11]:
characters = data_all.columns[2:8]
for character in characters:
    data_all[character] = data_all[character].apply(sent_segm)

**4. Работа с БД: создание и функция записи**

In [12]:
import sqlite3

In [13]:
conn = sqlite3.connect("./friends_corpus.db")
cur = conn.cursor()

In [14]:
cur.execute("""
CREATE TABLE IF NOT EXISTS Match 
(pr_key integer PRIMARY KEY, sentence_id integer, token_id integer)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Sentences
(sentence_id integer PRIMARY KEY, sentence text, annotated text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Meta
(sentence_id integer PRIMARY KEY, character text, episode integer, 
season integer, scriptwriters text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Tokens
(token_id integer PRIMARY KEY, token text, lemma_id integer)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS Lemmas
(lemma_id integer PRIMARY KEY, lemma text, pos text)""")

conn.commit()
conn.close()

In [15]:
def write_char_to_db(char_dict):
    conn = sqlite3.connect("./friends_corpus.db")
    cur = conn.cursor()
    
    cur.execute("INSERT INTO Sentences VALUES (?, ?, ?)", 
               (char_dict["sentence_id"], char_dict["sentence"], 
                char_dict["annotated"]))
    cur.execute("INSERT INTO Meta VALUES (?, ?, ?, ?, ?)", 
               (char_dict["sentence_id"], char_dict["character"], 
                char_dict["episode"], char_dict["season"],
                char_dict["scriptwriters"]))
    
    conn.commit()
    conn.close()

**5. Морфпарсинг и запись в БД размеченных предложений и мета инфы**

In [16]:
sent_id = 0

for character in characters: # для каждого персонажа
    i = 0
    for row in tqdm(data_all[character]): # для каждой серии
        season = data_all.loc[i, "season"]
        episode = data_all.loc[i, "episode"]
        scriptwriters = data_all.loc[i, "scriptwriters"]
        i += 1
        for sent in row.sents: # для каждого его/её предложения из этой серии
            s = sent.text.strip()
            if s: # если не пустая строка
                data_parsed = {}
                data_parsed["sentence"] = s
                s = nlp_pos(s) # тут другой спэйси, модельный
                s_ann = []
                for token in s:
                    s_ann.append('+'.join([token.text.lower(), token.lemma_, token.pos_]))
                data_parsed["annotated"] = ' '.join(s_ann)

                data_parsed["character"] = character
                data_parsed["season"] = int(season)
                data_parsed["episode"] = int(episode)
                data_parsed["scriptwriters"] = scriptwriters
                data_parsed["sentence_id"] = int(sent_id)
                sent_id += 1

                write_char_to_db(data_parsed)


100%|█████████████████████████████████████████| 227/227 [01:27<00:00,  2.59it/s]


**6. Таблицы лемм и токенов**

In [17]:
conn = sqlite3.connect("./friends_corpus.db")
df_annot = pd.read_sql_query("SELECT sentence_id, annotated FROM Sentences", conn)
conn.close()

In [18]:
df_annot.head()

,sentence_id,annotated
0,0,there+there+PRON 's+be+AUX nothing+nothing+PRO...
1,1,he+he+PRON 's+be+AUX just+just+ADV some+some+D...
2,2,"okay+okay+INTJ ,+,+PUNCT everybody+everybody+P..."
3,3,this+this+DET is+be+AUX not+not+PART even+even...
4,4,it+it+PRON 's+be+AUX just+just+ADV two+two+NUM...


In [19]:
tok_lem_pos_str = " ".join(df_annot["annotated"])
tok_lem_pos_set = set(tok_lem_pos_str.split(' '))

tok_lem_pos = []
lem_pos = []
for tok_i, tlp in enumerate(tok_lem_pos_set):
    tok, lem, pos = tlp.split("+")
    lempos = "+".join([lem, pos])
    tok_lem_pos.append((tok_i, tok, lempos, tlp))
    lem_pos.append((lempos, lem, pos))

In [20]:
df_tokens = pd.DataFrame(tok_lem_pos,
                         columns=["token_id", "token", "l+p", "t+l+p"])
df_tokens.head()

,token_id,token,l+p,t+l+p
0,0,perked,perk+VERB,perked+perk+VERB
1,1,ons,ons+NOUN,ons+ons+NOUN
2,2,dishes,dish+NOUN,dishes+dish+NOUN
3,3,effervesce,effervesce+VERB,effervesce+effervesce+VERB
4,4,igneous,igneous+ADJ,igneous+igneous+ADJ


In [21]:
df_lemmas = pd.DataFrame(lem_pos,
                         columns=["l+p", "lemma", "pos"])
df_lemmas.shape

(17626, 3)

In [22]:
df_lemmas = df_lemmas.drop_duplicates(ignore_index=True)
df_lemmas.shape

(14968, 3)

In [23]:
df_lemmas["lemma_id"] = df_lemmas.index
df_lemmas.head()

,l+p,lemma,pos,lemma_id
0,perk+VERB,perk,VERB,0
1,ons+NOUN,ons,NOUN,1
2,dish+NOUN,dish,NOUN,2
3,effervesce+VERB,effervesce,VERB,3
4,igneous+ADJ,igneous,ADJ,4


In [24]:
df_tokens_to_db = df_tokens.merge(df_lemmas, on="l+p").drop(["l+p", "t+l+p", "lemma", "pos"], axis=1)

In [25]:
df_tokens_to_db.head()

,token_id,token,lemma_id
0,0,perked,0
1,1,ons,1
2,2,dishes,2
3,6584,dish,2
4,3,effervesce,3


In [26]:
df_lemmas_to_db = df_lemmas.drop("l+p", axis=1)
df_lemmas_to_db.head()

,lemma,pos,lemma_id
0,perk,VERB,0
1,ons,NOUN,1
2,dish,NOUN,2
3,effervesce,VERB,3
4,igneous,ADJ,4


In [27]:
conn = sqlite3.connect("./friends_corpus.db")

df_tokens_to_db.to_sql("Tokens", con=conn, if_exists='append', index=False)
df_lemmas_to_db.to_sql("Lemmas", con=conn, if_exists='append', index=False)

conn.commit()
conn.close()

**7. Таблица мэтч**

In [28]:
df_annot.head()

,sentence_id,annotated
0,0,there+there+PRON 's+be+AUX nothing+nothing+PRO...
1,1,he+he+PRON 's+be+AUX just+just+ADV some+some+D...
2,2,"okay+okay+INTJ ,+,+PUNCT everybody+everybody+P..."
3,3,this+this+DET is+be+AUX not+not+PART even+even...
4,4,it+it+PRON 's+be+AUX just+just+ADV two+two+NUM...


In [29]:
match = []
for sent_id, annot in enumerate(list(df_annot["annotated"])):
    for tlp in annot.split(' '):
        match.append((sent_id, tlp))

df_match = pd.DataFrame(match, columns=["sentence_id", "t+l+p"])
df_match.head()

,sentence_id,t+l+p
0,0,there+there+PRON
1,0,'s+be+AUX
2,0,nothing+nothing+PRON
3,0,to+to+PART
4,0,tell+tell+VERB


In [30]:
df_match_to_db = df_match.merge(df_tokens, on="t+l+p").drop(["t+l+p", "token", "l+p"], axis=1)
df_match_to_db.head()

,sentence_id,token_id
0,0,6177
1,45,6177
2,46,6177
3,213,6177
4,214,6177


In [31]:
df_match_to_db = df_match_to_db.reset_index().rename(columns={"index": "pr_key"})

In [32]:
conn = sqlite3.connect("./friends_corpus.db")

df_match_to_db.to_sql("Match", con=conn, if_exists='append', index=False)

conn.commit()
conn.close()